In [ ]:
import re
import numpy as np
import joblib
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
import contractions
import spacy

In [ ]:
# Classifier
clf = joblib.load("../joblib/logistic_fake_review_model.pkl")

# TF-IDF + NN models
raw_tfidf = joblib.load("../joblib/raw_tfidf_vectorizer.pkl")
raw_nn = joblib.load("../joblib/raw_nn_model.pkl")

lemm_tfidf = joblib.load("../joblib/lemm_tfidf_vectorizer.pkl")
lemm_nn = joblib.load("../joblib/lemm_nn_model.pkl")

# Category centroids
category_centroids = joblib.load("../joblib/category_centroids.pkl")

# Numeric feature order
numeric_feature_order = joblib.load("../joblib/numeric_feature_order.pkl")

# NLP tools
sia = SentimentIntensityAnalyzer()
nlp = spacy.load("en_core_web_sm")

In [ ]:
def capital_letter_ratio(text):
    letters = [c for c in text if c.isalpha()]
    if not letters:
        return 0.0
    return sum(1 for c in letters if c.isupper()) / len(letters)

def punctuation_ratio(text):
    if not text:
        return 0.0
    return len(re.findall(r"[^\w\s]", text)) / len(text)

def repetition_score(text):
    words = re.findall(r'\b\w+\b', text.lower())
    if not words:
        return 0.0
    return 1 - len(set(words)) / len(words)

def expand_contractions(text):
    return contractions.fix(text) if isinstance(text, str) else ""

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    return re.sub(r"\s+", " ", text).strip()

def adjective_ratio(text):
    tokens = word_tokenize(text)
    if not tokens:
        return 0.0
    tags = pos_tag(tokens)
    return sum(1 for _, t in tags if t.startswith("JJ")) / len(tokens)

def sentiment_score(text):
    return sia.polarity_scores(text)["compound"] if text else 0.0

def rating_polarity(r):
    if r >= 4:
        return 1
    if r <= 2:
        return -1
    return 0

def rating_sentiment_mismatch(sentiment, rating):
    rp = rating_polarity(rating)
    if rp == 1 and sentiment < -0.2:
        return 1
    if rp == -1 and sentiment > 0.2:
        return 1
    return 0

def lemmatize(text):
    doc = nlp(text)
    return " ".join(t.lemma_ for t in doc if not t.is_space)

In [ ]:
def raw_review_similarity(text):
    vec = raw_tfidf.transform([text])
    distances, _ = raw_nn.kneighbors(vec, n_neighbors=5)
    sims = 1 - distances
    return sims[0].max()   

def lemm_review_similarity(text):
    vec = lemm_tfidf.transform([text])
    distances, _ = lemm_nn.kneighbors(vec, n_neighbors=5)
    sims = 1 - distances
    return sims[0].max() 

def category_consistency(text, category):
    if category not in category_centroids:
        return 0.0
    vec = lemm_tfidf.transform([text])
    centroid = category_centroids[category]
    return cosine_similarity(vec, centroid)[0][0]

In [ ]:
def predict_review(review, rating, category):
    # ---------- RAW ----------
    text_length = len(str(review))
    cap_ratio = capital_letter_ratio(review)
    punct_ratio = punctuation_ratio(review)
    rep_score = repetition_score(review)

    # ---------- CLEAN ----------
    expanded = expand_contractions(review)
    cleaned = clean_text(expanded)

    adj_ratio = adjective_ratio(cleaned)
    sent_score = sentiment_score(cleaned)
    mismatch = rating_sentiment_mismatch(sent_score, rating)
    extreme = 1 if rating in [1, 5] else 0

    # ---------- LEMMATIZED ----------
    lemm = lemmatize(cleaned)

    # ---------- STATEFUL ----------
    raw_sim = raw_review_similarity(review)
    lemm_sim = lemm_review_similarity(lemm)
    cat_score = category_consistency(lemm, category)

    # ---------- NUMERIC VECTOR ----------
    numeric_dict = {
        'text_length': text_length,
        'capital_ratio': cap_ratio,
        'punctuation_ratio': punct_ratio,
        'adjective_ratio': adj_ratio,
        'sentiment_score': sent_score,
        'rating_sentiment_mismatch': mismatch,
        'raw_review_similarity': raw_sim,
        'category_consistency_score': cat_score,
        'review_similarity_score': lemm_sim,
        'repetition_score': rep_score,
        'is_extreme_rating': extreme
    }

    numeric_values = np.array([[numeric_dict[f] for f in numeric_feature_order]])

    # ---------- FINAL MATRIX ----------
    X_text = lemm_tfidf.transform([lemm])
    X_final = hstack([X_text, numeric_values])

    # ---------- PREDICTION ----------
    pred = clf.predict(X_final)[0]
    prob = clf.predict_proba(X_final)[0, 1]

    return {
        "prediction": "Genuine" if pred == 1 else "Fake",
        "probability": float(prob)
    }

In [ ]:
predict_review(
    review="The phone overheats quickly and the battery drains too fast.",
    rating=2,
    category="Electronics"
)

In [ ]:
predict_review(
    review="Shoes looked nice but the sole started coming off after a week.",
    rating=1,
    category="Footwear"
)

In [ ]:
predict_review(
    review="This is the best product ever! Everyone must buy it!",
    rating=5,
    category="Accessories"
)

In [ ]:
predict_review(
    review="Amazing product, amazing quality, amazing experience!",
    rating=5,
    category="Beauty"
)

In [ ]:
predict_review(
    review="Highly recommend this to everyone. Very very good.",
    rating=5,
    category="Toys"
)

In [ ]:
predict_review(
    review="The product is very bad and I am extremely disappointed.",
    rating=2,
    category="Accessories"
)

In [ ]:
predict_review(
    review="This item broke quickly and feels cheap.",
    rating=1,
    category="Home Appliances"
)

In [ ]:
predict_review(
    review="Good good good product product product.",
    rating=5,
    category="Clothing"
)

In [ ]:
predict_review(
    review="Nice nice nice nice nice.",
    rating=4,
    category="Beauty"
)

In [ ]:
predict_review(
    review="Good product.",
    rating=4,
    category="Books"
)

In [ ]:
predict_review(
    review="Bad quality.",
    rating=1,
    category="Accessories"
)

In [ ]:
predict_review(
    review="The sound quality is crisp and bass is strong.",
    rating=5,
    category="Electronics"
)

In [ ]:
predict_review(
    review="The fabric is soft and stitching is neat.",
    rating=4,
    category="Clothing"
)

In [ ]:
predict_review(
    review="I absolutely hate this product. Worst purchase ever.",
    rating=1,
    category="Accessories"
)

In [ ]:
predict_review(
    review="Terrible experience. Completely useless item.",
    rating=1,
    category="Home Appliances"
)

In [ ]:
predict_review(
    review="The battery dies very soon and it barely works for 20 minutes",
    rating=2,
    category="Home Appliances"
)